# Web scraping

In [1]:
import pandas as pd

# Import for the Translation with IBM Watson
import requests
import os
import json
import getpass
import ast # convert str to dic

# Imports the Google Cloud client library
# from googletrans import Translator

In [33]:
supermarkets_en = {
    #'aldi_north': pd.read_csv('data/products-en/aldinorth-products-en.csv'),
    #'aldi_south': pd.read_csv('data/products-en/aldisouth_products-en.csv'),
    'edeka': pd.read_csv('data/products-en/edeka-products-en.csv'),
    'kaufland': pd.read_csv('data/products-en/kaufland-products-en.csv'),
    'lidl': pd.read_csv('data/products-en/lidl-products-en.csv'),
    'rewe': pd.read_csv('data/products-en/rewe-product-en.csv')
}

ParserError: Error tokenizing data. C error: Expected 7 fields in line 178, saw 9


Now that we have product info of the main 6 supermarkets in Berlin, and before we use it we are going to explore the data.

In [3]:
supermarkets['rewe']

,Unnamed: 0,Name,Price,Unit,Packet Size,Supermarket,Comparable Price
0,0,Ja! Briespitze 60% Fett,1.19,100 Gramm,Packung 200 Gramm,rewe,0.600
1,1,REWE Beste Wahl Brie 45% Fett,0.69,100 Gramm,Packung 100 Gramm,rewe,0.690
2,2,REWE Beste Wahl Ziegenfrischkäse Mousse 73%,1.99,100 Gramm,Becher 125 Gramm,rewe,1.590
3,3,Rewe Brie 45% Fett,0.79,100 Gramm,Packung 100 Gramm,rewe,0.790
4,4,Nutella,3.79,100 Gramm,Glas 750 Gramm,rewe,0.505
...,...,...,...,...,...,...,...
6238,6238,ja! Kartoffelringe Paprika,0.99,100 Gramm,Beutel 100 Gramm,rewe,0.990
6239,6239,ja! Schoko Rosinen,0.99,100 Gramm,Beutel 200 Gramm,rewe,0.500
6240,6240,Ja! Erdnüsse Paprika,0.81,100 Gramm,Beutel 200 Gramm,rewe,0.410
6241,6241,ja! Orange Cake,0.99,100 Gramm,Packung 300 Gramm,rewe,0.330


In [4]:
supermarkets['rewe'].dtypes

Unnamed: 0            int64
Name                 object
Price               float64
Unit                 object
Packet Size          object
Supermarket          object
Comparable Price    float64
dtype: object

In [5]:
supermarkets['rewe']['Name'].value_counts()

Ja! Champignons 3. Wahl - geschnitten         3
Ja! Tomaten Ketchup                           3
Dr. Oetker Die Ofenfrische Pizza Speciale     2
REWE Feine Welt Chardonnay                    2
ja! Kleiner Käse für den Ofen mild            2
                                             ..
Dr. Oetker Tortenguss klar gezuckert          1
Kerrygold Original Irische Butter gesalzen    1
Nadler Gartensalat Buntes Gemüse              1
Nescafe Gold entkoffeiniert                   1
REWE Beste Wahl Himbeeren                     1
Name: Name, Length: 6194, dtype: int64

In [6]:
for market in supermarkets:
    # Drop duplicates within the 'Name' column
    supermarkets[market] = supermarkets[market].drop_duplicates('Name')
    # Drop 'Unnamed: 0' column
    supermarkets[market] = supermarkets[market].drop(columns='Unnamed: 0', errors='ignore')

In [7]:
supermarkets['rewe']['Name'].str.split(',')

0                           [Ja! Briespitze 60% Fett]
1                     [REWE Beste Wahl Brie 45% Fett]
2       [REWE Beste Wahl Ziegenfrischkäse Mousse 73%]
3                                [Rewe Brie 45% Fett]
4                                           [Nutella]
                            ...                      
6238                     [ja! Kartoffelringe Paprika]
6239                             [ja! Schoko Rosinen]
6240                           [Ja! Erdnüsse Paprika]
6241                                [ja! Orange Cake]
6242           [ja! Kalifornische Pistazien geröstet]
Name: Name, Length: 6194, dtype: object

Once we have the data clean, we will translate it from German to English

## 2. Translate

We are goin to use the googletrans module in order to translate our .csv

### 2.1 IBM Watson Translate API

In [8]:
# yaotT_lJIzyTXCRT9_POnyQqBLprpP6u9guac38WweEc
api_key = getpass.getpass(prompt='IBM API key: ')

IBM API key: ········


In [10]:
url = "https://api.eu-de.language-translator.watson.cloud.ibm.com/instances/8218aa64-0979-48a3-9395-cefb38349b8e/v3/translate?version=2018-05-01"
headers = {"Content-Type": "application/json"}
auth = ("apikey", api_key)

In [11]:
test = 'Pfanner Mango Fair 100%  Fair'
data = {"text":test,"model_id":"de-en"}

In [12]:
json.dumps(data)

'{"text": "Pfanner Mango Fair 100%  Fair", "model_id": "de-en"}'

In [13]:
# Request to the IBM Watson API
r = requests.post(headers=headers, auth=auth, url=url, data=json.dumps(data))

In [14]:
r.json()

{'translations': [{'translation': 'Pfanner Mango Fair 100% Fair'}],
 'word_count': 6,
 'character_count': 29}

In [15]:
r.json()['translations'][0]['translation']

'Pfanner Mango Fair 100% Fair'

In [16]:
ast.literal_eval(json.dumps(data))['text']

'Pfanner Mango Fair 100%  Fair'

IBM has a limitation of 'Exceeded max content length for translate request, max allowed is 50 KiB.'
So, in order to translate the whole csv file, we will have to go through each cell and translate it. We only need to translate 'Name', 'Unit' and 'Packet Size'.

In [17]:
# Create an empty DataFrame for the tranlated ingredients with the 'Name' as key
supermarkets_example = {}
supermarkets_example['rewe-en'] = pd.DataFrame(columns=('Name', 'Name-en'))

In [18]:
supermarkets_example['rewe-en']

,Name,Name-en


In [19]:
data = {"text":['Wasser'],"model_id":"de-en"}

json.dumps(data)

r = requests.post(headers=headers, auth=auth, url=url, data=json.dumps(data))
           
new_row = {'Name': ast.literal_eval(json.dumps(data))['text'][0], 'Name-en': r.json()['translations'][0]['translation']}

In [20]:
new_row

{'Name': 'Wasser', 'Name-en': 'Water'}

In [21]:
supermarkets_example['rewe-en'] = supermarkets_example['rewe-en'].append(new_row, ignore_index=True)

In [22]:
supermarkets_example['rewe-en']

,Name,Name-en
0,Wasser,Water


In [23]:
[supermarkets['rewe']['Name'][1]]

['REWE Beste Wahl Brie 45% Fett']

In [24]:
src = 'de' # Source language
dest = 'en' # Destiny language

supermarkets_en = {
    'aldi_north': pd.DataFrame(columns=('Name', 'Name-en')),
    'aldi_south': pd.DataFrame(columns=('Name', 'Name-en')),
    'edeka': pd.DataFrame(columns=('Name', 'Name-en')),
    'kaufland': pd.DataFrame(columns=('Name', 'Name-en')),
    'lidl': pd.DataFrame(columns=('Name', 'Name-en')),
    'rewe': pd.DataFrame(columns=('Name', 'Name-en'))
}

In [25]:
for market in supermarkets:
    # Create new dataframe for the english version
    # supermarkets[f'{market}-en'] = supermarkets[market].copy()

    # Loop through each row
    for row in range(0,5):# range(len(supermarkets[market])):
        try:
            data = {"text":[supermarkets[market]['Name'][row]], "model_id":"de-en"}
            r = requests.post(headers=headers, auth=auth, url=url, data=json.dumps(data))
            # Create a new row with the word in 'de' and in 'en'
            new_row = {'Name': ast.literal_eval(json.dumps(data))['text'][0], 'Name-en': r.json()['translations'][0]['translation']}
            supermarkets_en[market] = supermarkets[market].append(new_row, ignore_index=True)

        except:
            print(supermarkets[market]['Name'][row])

In [26]:
supermarkets_en['rewe']

,Name,Price,Unit,Packet Size,Supermarket,Comparable Price,Name-en
0,Ja! Briespitze 60% Fett,1.19,100 Gramm,Packung 200 Gramm,rewe,0.600,NaN
1,REWE Beste Wahl Brie 45% Fett,0.69,100 Gramm,Packung 100 Gramm,rewe,0.690,NaN
2,REWE Beste Wahl Ziegenfrischkäse Mousse 73%,1.99,100 Gramm,Becher 125 Gramm,rewe,1.590,NaN
3,Rewe Brie 45% Fett,0.79,100 Gramm,Packung 100 Gramm,rewe,0.790,NaN
4,Nutella,3.79,100 Gramm,Glas 750 Gramm,rewe,0.505,NaN
...,...,...,...,...,...,...,...
6190,ja! Schoko Rosinen,0.99,100 Gramm,Beutel 200 Gramm,rewe,0.500,NaN
6191,Ja! Erdnüsse Paprika,0.81,100 Gramm,Beutel 200 Gramm,rewe,0.410,NaN
6192,ja! Orange Cake,0.99,100 Gramm,Packung 300 Gramm,rewe,0.330,NaN
6193,ja! Kalifornische Pistazien geröstet,3.79,100 Gramm,Beutel 250 Gramm,rewe,1.520,NaN


In [ ]:
data = {"text":test,"model_id":"de-en"}
json.dumps(data)

# Request to the IBM Watson API
r = requests.post(headers=headers, auth=auth, url=url, data=json.dumps(data))

r.json()

r.json()['translations'][0]['translation']

The method accepts lists as input as well. So we are going to loop per each row in order to catch the rows that raise errors

---------------

In [ ]:
def translate(dataframe):
    # We need to translate 'Name', 'Unit', 'Packet Size'
    supermarkets[f'{dataframe}-en']

In [ ]:
# Instantiates a client
translator = Translator()

In [ ]:
# Example to see how it works
# 1 - translator.translate('Hello').text

In [ ]:
# 1 - word.origin # .origin -> prints the original word

In [ ]:
# 1 - word[0].text # .text -> prints the translated word

In [ ]:
# Undertand how the method works
# 1 - translator.translate(supermarkets['rewe']['Name'][1], dest='en').text

## 3. Next Steps

- Create a dictionary with a dataframe per retailer with unique ingredients 
- [v0] - Build an algorithm able to (input: ingredient, output: % coverage + total price per each supermarket)
- [v1] - Build an algorithm able to (input: list of ingredients, output: % coverage + total price per each supermarket)
- [v2] - Use fuzzy buzzy to match the input (potato) with the ingredients by similarity.

### 3.2 [v0] algorithm
- [v0] - Build an algorithm able to (input: ingredient, output: % coverage + total price per each supermarket)

In [ ]:
# We are looking for the price of the ingredient that we are searching per each supermarket
supermarkets['rewe'][supermarkets['rewe']['Name'] == 'Pfanner Mango Fair 100%  Fair']['Price'].values[0]

In [ ]:
def search(ingredient:str):
    # New empty list
    result = []
    for market in supermarkets:
        a = supermarkets[market]
        try:
            price = a[a['Name'] == ingredient]['Price'].values[0]
            result.append(f'{market}: {price}')
        except:
            result.append(f'{market}: Not found')
    return result

# Try it with 'Coca Cola' because it is available in every supermarket
# Try with 'Pfanner Mango Fair 100%  Fair'

In [ ]:
# Testing the function
search('Pfanner Mango Fair 100%  Fair')

In [ ]:
# Testing the function
search('Coca Cola')

We can se that there are some inconsistencies with the 'Coca Cola' prices. Let's find out which is the problem

In [ ]:
supermarkets['kaufland'][supermarkets['kaufland']['Name'] == 'Coca Cola']

### 3.3 [v1] algorithm
- [v1] - Build an algorithm able to (input: list of ingredients, output: % coverage + total price per each supermarket)

In [ ]:
def search_list(ingredients:list):
    # New empty list
    result = []
    
    for market in supermarkets:
        
        a = supermarkets[market]
        
        for ing in ingredients:    
            try:
                price = a[a['Name'] == ing]['Price'].values[0]
                result.append(f'{market}: {price} for the {ing}')
            except:
                result.append(f'{market}: Not found')
            
    return result

In [ ]:
search_list(['Coca Cola', 'Pfanner Mango Fair 100%  Fair'])

## Clean data

- Translate data to english
- Isolate the package size
- Check values
- Normalize Unit

- Create a unique list with unique products of all different markets
- Analyse it
- Conclude how to approach the solution

There are inconsistencies in the sizes of the packages. Let's normalize the package size to 100 (ml, g, ...). To do it, we have to get the unit from the package size and: [price / unit * 100]